# Nuz Klassifir

### Download NLTK packages

In [241]:
import nltk
#nltk.download()

### Training Data Collection and Preprocessing

In [242]:
import os, re
direk_folders = ['culture', 'diverse', 'economy', 'internationalNews', 'localNews', 'politic', 'society', 'sport', 'technology' ]

In [243]:
train = []

for folder_name in direk_folders:
    direk = 'train/' + folder_name + '/'
    current_files_names = os.listdir(direk)
    
    i = 0
    for file_name in current_files_names:
        
        i +=1
        if i == 100:
            break
            
        file_reader = open(direk + file_name, "r", encoding="utf-8")
        file = file_reader.read()

        # lowercase
        #text=text.lower()

        #remove tags
        #text=re.sub("<!--?.*?-->","",text)

        # remove special characters and digits
        #text=re.sub("(\\d|\\W)+"," ",text)

        title = re.search(r'<TITLE>(.*?)</(.?)TITLE>', file)
        cleanr = re.compile('<.*?>')
        clean_title = re.sub(cleanr, '', title.group(0))

        # TODO include TEXT tag in the input

        train.append([clean_title, folder_name])

### Stemming

In [244]:
from nltk.stem.porter import *
from nltk import word_tokenize
#nltk.download('punkt')

In [245]:
for row in train:
    #tokenize
    splitted = word_tokenize(row[0])

    for i in range(0, len(splitted)):
        splitted[i] = stemmer.stem(splitted[i])
    
    row[0] = splitted

### Stop-word removal

In [246]:
#nltk.download('stopwords')
from nltk.corpus import stopwords

In [247]:
stop_words = set(stopwords.words('arabic')) 
for row in train:
    x = [w for w in row[0] if not w in stop_words]
    row[0] = x

### URL removal

In [248]:
for row in train:
    splitted = row[0]
    row[0] = " ".join(splitted)
    row[0] = re.sub(r'http\S+', '', row[0])

### Model

In [249]:
from sklearn.feature_extraction.text import TfidfVectorizer

train_sent = [line[0] for line in train]
train_labels = [line[1] for line in train]

tdif_vec = TfidfVectorizer(norm = None)
tfidf = tdif_vec.fit_transform(train_sent)

### Feed the training documents and their labels to a NB classifier

In [250]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB(alpha=1)
clf.fit(tfidf, train_labels)

MultinomialNB(alpha=1, class_prior=None, fit_prior=True)

### Test Data Collection and Preprocessing

In [252]:
import json
json_file = open('test\posts.json', encoding='utf8')
json_reader = json.load(json_file)

user_interest = []

test = []
for user in json_reader.values():
    for article in user:
        #use the title and description as input
        test.append(article[0]+' '+ article[1])

# number of data provided for each user is 50, nubmer of users is 2000
test = test[:100]

### Stemming

In [253]:
for i in range(0, len(test)):
    #tokenize
    splitted = word_tokenize(test[i])

    #stem
    for j in range(0, len(splitted)):
        splitted[j] = stemmer.stem(splitted[j])
    
    test[i] = splitted

### Stop-word removal

In [254]:
for i in range(0, len(test)):
    x = [w for w in test[i] if not w in stop_words]
    test[i] = x

### URL removal

In [255]:
for i in range(0, len(test)):
    splitted = test[i]
    cell = " ".join(splitted)
    test[i] = re.sub(r'http\S+', '', cell)

### Predict the labels of the testing data

In [256]:
transformed_test = tdif_vec.transform(test)
predictions = clf.predict(transformed_test)
print(predictions)

['localNews' 'economy' 'technology' 'internationalNews' 'politic'
 'culture' 'technology' 'technology' 'technology' 'internationalNews'
 'internationalNews' 'internationalNews' 'diverse' 'sport' 'sport' 'sport'
 'sport' 'culture' 'localNews' 'politic' 'sport' 'politic' 'technology'
 'diverse' 'sport' 'economy' 'technology' 'economy' 'culture' 'localNews'
 'economy' 'culture' 'technology' 'culture' 'sport' 'localNews' 'culture'
 'localNews' 'localNews' 'society' 'culture' 'localNews' 'sport' 'culture'
 'technology' 'economy' 'culture' 'sport' 'diverse' 'culture' 'localNews'
 'economy' 'culture' 'localNews' 'politic' 'culture' 'economy' 'sport'
 'society' 'technology' 'localNews' 'localNews' 'economy' 'sport'
 'technology' 'culture' 'sport' 'society' 'localNews' 'economy' 'culture'
 'sport' 'technology' 'culture' 'society' 'technology' 'localNews'
 'diverse' 'culture' 'culture' 'politic' 'culture' 'diverse' 'localNews'
 'sport' 'culture' 'localNews' 'technology' 'diverse' 'culture'
 'int

### Calculate each user interests

In [257]:
articles_per_user = 50
interests = [{}]
user_index = 0
for i in range(0, len(predictions)):
    if predictions[i] not in interests[user_index].keys():
        interests[user_index][predictions[i]] = 0
    interests[user_index][predictions[i]] = interests[user_index][predictions[i]] + 1
    
    if i != 0 and i % articles_per_user == 0:
        user_index += 1
        interests.append({})
        
user_index = 0
for user in interests:
    user_interests = ('User {}:').format(user_index)
    for key, value in user.items():
        user_interests = ('{} {} {}%,').format(user_interests, key, (value*100.00/articles_per_user))
    user_index += 1
    print(user_interests)

User 0: localNews 16.0%, economy 10.0%, technology 16.0%, internationalNews 8.0%, politic 6.0%, culture 20.0%, diverse 6.0%, sport 18.0%, society 2.0%,
User 1: economy 8.0%, culture 24.0%, localNews 18.0%, politic 4.0%, sport 12.0%, society 6.0%, technology 16.0%, diverse 8.0%, internationalNews 2.0%,
